# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd
import re
import numpy as np
# nltk
import nltk
nltk.download('stopwords')
nltk.download('wordnet') # download for lemmatization
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
# sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
# from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
# from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

[nltk_data] Downloading package stopwords to /home/martin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/martin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/martin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterData.db')
df = pd.read_sql_table('TextMessages', engine)
X = df[["message", "original", "genre"]]
Y = df.drop(columns= ["id", "message", "original", "genre"])

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    # Normalization
    
    # Convert to lower case
    text = text.lower()
    
    # Remove punctuation characters - this regex finds everything which is not a combination of letters
    # and numbers and replaces it with a whitespace
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    
    
    # Tokenization
    
    # Split into tokens
    words = word_tokenize(text)
    
    
    # Remove stopwords
    words = [w for w in words if w not in stopwords.words("english")]
    
    # Part-of-speech tagging maybe useful here?
    # Named Entity Recognition usefuk here?
    
    # Stemming - only keep the stem of a word, simple find and replace method which removes f.e. "ing"
    # stemmed = [PorterStemmer().stem(w) for w in words]
    
    # Lemmatization - more complex appraoch using dictionaries which can f.e. map "is" and "was" to "be"
    # Lemmatize verbs by specifying pos
    lemmed_verbs = [WordNetLemmatizer().lemmatize(w, pos='v') for w in words]
    # Reduce nouns to their root form
    lemmed_nouns = [WordNetLemmatizer().lemmatize(w) for w in lemmed_verbs]
    return lemmed_nouns

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
# pipeline = Pipeline([
#         ('features', FeatureUnion([

#             ('text_pipeline', Pipeline([
#                 ('vect', CountVectorizer(tokenizer=tokenize)),
#                 ('tfidf', TfidfTransformer())
#             ]))
#         ])),

#         ('clf', MultiOutputClassifier(RandomForestClassifier()))
#     ])

pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size = 0.2)

In [8]:
pipeline.fit(X_train["message"], y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7f6da66efd30>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
y_pred = pipeline.predict(X_test["message"])
columns = y_test.columns
y_pred_df = pd.DataFrame(y_pred, columns = columns)

In [ ]:
# Calculate the average accuracy for each target column
def print_acc(print_text, y_test, y_pred)
    y_pred_df = pd.DataFrame(y_pred_cv, columns = columns)
    accuracy = (y_pred_df == y_test.reset_index().drop(["index"], axis = 1)).mean()
    print(print_text)
    print(f"Average accuracy: {accuracy.mean()}")
    print(accuracy)

In [30]:
print_acc("Accuracy per category before GridSearch: ", y_test, y_pred)

related                   0.809708
request                   0.890097
offer                     0.995470
aid_related               0.767272
medical_help              0.918562
medical_products          0.951128
search_and_rescue         0.972155
security                  0.981977
military                  0.967577
water                     0.950126
food                      0.935822
shelter                   0.930816
clothing                  0.985410
money                     0.977447
missing_people            0.988557
refugees                  0.966624
death                     0.954275
other_aid                 0.869547
infrastructure_related    0.933200
transport                 0.954704
buildings                 0.951604
electricity               0.980546
tools                     0.993563
hospitals                 0.988175
shops                     0.995661
aid_centers               0.988032
other_infrastructure      0.954894
weather_related           0.879702
floods              

# Comments

- related has 0,1 and 2 as a value -> Convert all values to 0 (0) or 1 (1, 2)

- child_alone only has 0 as a value -> is this caused by a unlucky train-test split or is this true for every data row? -> Yes this is true for all data rows -> we drop this row earlier

- We focus on the f1-score because we have no further specification if false positives or false negatives should be especially avoided

- the overall f1 score per column (avg / total) is quite good, but for some options which are very rate, the f1 score is quite low

### 6. Improve your model
Use grid search to find better parameters. 

In [14]:
# Check for available parameters to optimize
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__ccp_alpha', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__max_samples', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf_

In [15]:
# pipeline = Pipeline([
#         ('vect', CountVectorizer(tokenizer=tokenize)),
#         ('tfidf', TfidfTransformer()),
#         ('clf', MultiOutputClassifier(RandomForestClassifier()))
#     ])

parameters = {
    # vect
    
    # tfidf
    # https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html
    'tfidf__norm' : ['l1', 'l2'],
    
    # clf
    # https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
    'clf__estimator__criterion' : ['gini', 'entropy']
    

}

cv = GridSearchCV(pipeline, param_grid=parameters) 
cv.fit(X_train["message"], y_train)
y_pred_cv = cv.predict(X_test)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
print_acc("Accuracy per category after GridSearch: ", y_test, y_pred_cv)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
pipeline_2 = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.